# Ejercicio Complementario 3 - Reducción Dimensional con PCA

Este ejercicio se centra en el análisis de la relevancia de diferentes atributos de un conjunto de datos de campañas de marketing de productos bancarios, buscando la reducción de los atributos, dejando los esenciales para un posterior modelo predictivo.

## Contexto: Análisis de éxito en campaña de marketing

Fuente: https://archive.ics.uci.edu/ml/datasets/Bank+Marketing

El foco está en la implementación de una red neuronal para clasificación y la comparación de su desempeño con otro modelo de clasificación para predicir el valor de un atributo, desde un *dataset* de información de un resultados de personas contactadas por una campaña de marketing y que compraron la oferta (atributo "OK"), con cerca de 41.200 registros de personas contactadas.

Este conjunto de datos (abierto para este tipo de usos instruccionales), consiste en 20 atributos y 1 clase de etiquetas (totalizando 21 columnas) y corresponde a los datos de una campaña telefónica a diversos clientes en Portugal, ofreciéndoles la compra de un producto bancario. En varios casos, un cliente fue contactado varias veces antes de aceptar el el depósito a plazo ofrecido por la campaña (OK = yes).

Algunos de los atributos relevantes son (combinando atributos categóricos, con numéricos):
* **Datos personales**: Edad, Ocupación, Estado Civil, Nivel de Educación.
* **Datos financieros**: Su casa tiene crédito hipotecario, default: si el crédito ha caído en quiebra; tiene un crédito de consumo.
* **Datos de contactos de la campaña actual**: Tipo de Comunicación (celular o teléfono fijo); Mes del último contacto; Día de la semana del contacto; duración de la llamada (segundos); Contacto: N° de contactos durante la campaña; DíasAtrás: días transcurridos desde último contacto; Resultado: resultado de la última llamada (falló, no-existe, éxito)
* **Datos socioeconómicos**: EmpTasaVar: tasa de variación de empleabilidad; IPC: índice de precios consumidor mensual; ICC: índice de confianza consumidor mensual; Euribor3m: tasa euribor de 3 meses indicador diario; NumEmpleados: cantidad de gente empleada, en indicador trimestral.

Esta adaptación en particular, por el equipo de R:Solver (RSolver.com), enfrenta diferentes objetivos de aprendizaje dentro de los cursos de Big Data y Machine Learning.



## Instrucciones Generales
En este caso, se busca entender el comportamiento y desempeño de un modelo de clasificación sobre una versión compactada o de dimensión reducidad del dataet original, para predecir la variable de interés: **OK**, que servirá para predecir en casos futuros, según los datos de contactabilidad de un cliente, si el cliente aceptará o no contratar el depósito a plazo. Por ello, se recomienda seguir los pasos y buscar la información para contestar las preguntas.

**Este ejercicio es sin nota: no se envía informe**



## Paso 0: Instalar las librerías de modelos y funciones necesarias

Esto se ejecuta sólo una vez al comienzo de la sesión de cada persona. No se necesita volver a ejecutar con cada nueva prueba del resto de los scripts.

In [ ]:
install.packages('e1071')
install.packages('caret')
install.packages('randomForest')
install.packages('mass')
install.packages('ggfortify')


## Paso 1: Carga de los datos

La siguiente celda de código, carga los datos desde la URL de origen y luego muestra un encabezado con las primeras filas del dataset, para demostrar la disponibilidad de los datos. Nótese que quedan dos datasets: marketing.reduced tiene los datos originales, menos algunas columnas y mejor balanceado en clases, y marketing.pca queda para utilizar en el análisis PCA.

Esto también puede ser ejecutado una sola vez, si es conveniente.

In [ ]:
# Se declara la URL de dónde obtener los datos
theUrlMain <- "http://www.rsolver.com/dcc/docs/bank-additional-full-numbers.csv"

# Se declaran los nombres de las columnas
columnas = c("Edad","Ocupación","EstadoCivil","Educación","Default","Hipotecario","Consumo","Contacto","Mes","Día",
             "Duración","NumContactos","DíasAtrás","Previo","ResultadoPrevio", "EmpTasaVar", "IPC", "ICC", "Euribor3m", "NumEmpleados", "OK")

# Se cargan datos principales a una estructura (marketing.data), asignando nombres de atributos a las columnas
marketing.data <- read.table(file = theUrlMain, header = TRUE, sep = ";", col.names = columnas, na.strings = c("unknown","NA"))

# Se eliminan los registros que tienen algún NA (unknown)
marketing.clean <- na.omit(marketing.data) # Sólo quedan poco más de 30.000 filas (de las 41.000 originales)

# Aquí se arman dos subconjuntos con los datos de cada una de las dos clases.
# Se pueden ver los respectivos tamaños al terminar, evidenciando un desbalance.
clean.data.YES <- marketing.clean[marketing.clean$OK == 1,]
clean.data.NO <- marketing.clean[marketing.clean$OK == 0,]
dim(clean.data.YES) # Se ve que este es el conjunto más pequeño
dim(clean.data.NO)

# Se balancean las clases para entrenar: se busca acercar la cantidad de ejemplos positivos, con los negativos.
# Para esto se puede definir la cantidad de ejemplos de la clase más abundante (NO)
# en una proporción (balance_ratio) de la cantidad de registros de la clase menos abundante (YES)
# balance_ratio = 1.0 implica la misma cantidad para NO y para YES. Según haber probado, se puede elegir un número distinto de 1
balance_ratio <- 1.0

clean.subdata.YES <- clean.data.YES  # No se aplica sample(): se usan todos los ejemplos de la clase OK (que es la que tiene menos ejemplos)
clean.subdata.NO <- clean.data.NO[sample(nrow(clean.data.NO), balance_ratio*dim(clean.data.YES)[1]), ]

# Muestra cantidad de ejemplos contenidos en cada subconjunto
dim(clean.subdata.YES)
dim(clean.subdata.NO)

# Se juntan para el conjunto de referencia, ahora más balanceado
clean.subdata <- rbind(clean.subdata.YES, clean.subdata.NO)

# Se genera un dataset de trabajo (marketing.reduced) que no considera las columnas categóricas,
# para trabajar sólo con columnas numéricas
marketing.reduced <- clean.subdata
marketing.reduced$Default <- NULL
marketing.reduced$Ocupación <- NULL
marketing.reduced$EstadoCivil <- NULL
marketing.reduced$Educación <- NULL
marketing.reduced$Contacto <- NULL
marketing.reduced$Mes <- NULL
marketing.reduced$Día <- NULL
marketing.reduced$ResultadoPrevio <- NULL

# Nótese la existencia de 2 datasets:
# uno con la variable de interés OK (reduced) y el otro sin esa variable (pca)
marketing.pca <- marketing.reduced[,1:12]

# Se muestran las primeras líneas del dataset, incluyendo nombres asignados a las columnas
head(marketing.pca, 10)

# Se muestra un resumen de las columnas del dataset, describiendo los datos que se encuentran
summary(marketing.reduced)


## Paso 2: Uso de Random Forest para clasificación de referencia

Esta ejecución usa un Random Forest sobre el dataset original, ya balanceado, pero con todas las dimensiones definidas, antes de aplicar PCA, de modo de poder compararlo.

In [ ]:
working.data <- marketing.reduced

# Ahora se configuran los conjuntos de entrenamiento y testing en una proporción
# (por ej: 0.70 = 70% para training y el resto para evaluación o testing)
ratio = sample(1:nrow(working.data), size = 0.70*nrow(working.data))
training.data = working.data[ratio,]
testing.data = working.data[-ratio,]

library(randomForest)
library(caret)

# Random Forest
RF_model <- randomForest(as.factor(OK) ~ ., data=training.data, method="class")
RF_predict <- predict(RF_model, testing.data, type = "class")
confusionMatrix(RF_predict, as.factor(testing.data$OK), positive = '1')

## Paso 3: Test de relevancia por p-value

Aquí es factible observar las variables o atributos con p-value > 0.05, es decir, con relación a un nivel de significancia de 0.05.

Entonces, aquellos atributos o variables con un valor mucho menor que 0.05 son consideradas relevantes.

**Preguntas a responder**

Desde el análisis por relevancia, ¿cuántos y cuáles deberían ser los atributos significativos de considerar un posible mapeo a otras dimensiones (componentes principales)? ¿Por qué?


In [ ]:
model <- lm(formula = OK ~., data = marketing.reduced)
summary(model)


## Paso 4: Análisis PCA

A continuación se calculan los principales componentes desde los datos, gracias a la función prcomp(). En estas líneas, se ejecutan y muestra lo siguiente:

*   prcomp(marketing.pca, center = TRUE, scale. = TRUE): construye los componentes principales. center = TRUE los calcula de forma tal que el promedio calza en el centro del modelo. scale. = TRUE realiza una normalización numérica (todos los atributos en la misma escala numérica) antes de realizar el análisis.
*   summary(pca_result): muestra el comportamiento numérico de cada uno de los componentes principales (PC) según cada uno de los 14 atributos analizados. Se puede ver que se muestra la desviación estándar, pero más importante, se muestra la proporción de la varianza. Se nota que PC1 tiene una proporción de varianza mayor que el resto.
*   plot(pca_result, type='l'): muestra los mismos datos en forma de gráfico lineal, demostrando que el primer PC es el que tiene mayor proporción de varianza comparado con los otros.
*   print(pca_result$rotation): muestra la composición o pesos de cada atributo en relación al respectivo PC. Nótense aquellos casos en que los pesos son relativamente equivalentes entre ellos.

**Preguntas y Análisis**

Dado el análisis de PCA de la sección anterior, ¿cuáles de los PC valdría la pena considerar y - eventualmente - cuáles no, para un nuevo proceso de clasificación supervisada?

De aquí se construye el nuevo dataset, para entrenamiento y evaluación, considerando sólo los atributos (PC) que se determine sean significativos, apuntando así, a un proceso de clasificación más confiable, de entrenamiento con menor esuferzo y potencialmente menos propenso al overfitting.


In [ ]:
library(ggfortify)

# Se construye el modelo PCA
pca_result <- prcomp(marketing.pca, center = TRUE, scale. = TRUE)

# Análisis de comportamiento numérico o importancia numérica de cada PC
summary(pca_result)

# Composición (pesos) de cada PC en relación a los 14 atributos
print(pca_result$rotation)

# Valores propios de los datos originales
print('Eigen Values')
eigen(cor(marketing.reduced))$values

# Gráfico de línea que muestra la proporción de varianza de cada PC
plot(pca_result, type='l')

autoplot(pca_result)


## Parte 4: Preparación conjuntos de entrenamiento y evaluación

El siguiente código prepara los datos para el entrenamiento y evaluación. Se pueden probar diferentes proporciones de entrenamiento vs. evaluación, pero toso están trabajando sobre el dataset reducido en dimensiones, gracias al proceso PCA.

**Pregunta a responder y aplicar**

¿Cuáles deberían ser los PC a mantener en el dataset y que den buenos resultados en un modelo entrenado?

Nótese que cualquier reducción de las 13 columnas originales implica un menor tiempo de entrenamiento del modelo, pero no necesariamente mejor desempeño.

In [ ]:

# Proporción Training/Testing. Por ej: 0.75 = 75% entrenamiento y 25% validación
ratio = sample(1:nrow(marketing.reduced), size = 0.75*nrow(marketing.reduced))
trainingSet = marketing.reduced[ratio,]
testingSet = marketing.reduced[-ratio,]

# Aquí se construyen los nuevos training y testing set, pero sólo con datos mapeados de los PCA
trSet <- predict(pca_result, trainingSet)
trSet <- data.frame(trSet, trainingSet$OK)
teSet <- predict(pca_result, testingSet)
teSet <- data.frame(teSet, testingSet$OK)

# Si no todos los PCA son relevantes, se pueden dejar sólo aquellos más influyentes.
# A continuación hay instrucciones para eliminar algunos PC, dejando el resto.
# Se puede probar y ver qué combinación produce mejores resultados en el RF
trSet$PC12 <- NULL
trSet$PC11 <- NULL
trSet$PC10 <- NULL
#trSet$PC9 <- NULL
#trSet$PC8 <- NULL
#trSet$PC7 <- NULL
#trSet$PC6 <- NULL
#trSet$PC5 <- NULL
#trSet$PC4 <- NULL
#trSet$PC3 <- NULL
#trSet$PC2 <- NULL

teSet$PC12 <- NULL
teSet$PC11 <- NULL
teSet$PC10 <- NULL
#teSet$PC9 <- NULL
#teSet$PC8 <- NULL
#teSet$PC7 <- NULL
#teSet$PC6 <- NULL
#teSet$PC5 <- NULL
#teSet$PC4 <- NULL
#teSet$PC3 <- NULL
#teSet$PC2 <- NULL


head(trSet, 20)
head(teSet, 20)

## Parte 5: Implementación de un modelo de clasificación para ver su desempeño sobre el problema de dimensión reducida

Esta sección implementa un Árbol de Decisión, como modelo de clasificación de referencia, recordando que en los ejercicios anteriores, este árbol de decisión logró un Accuracy cercano al 91% en las mejores condiciones. La pregunta que se plantea para este ejercicio completo es si - gracias a una reducción dimensional por medio de PCA - se logra mejorar ese desempeño.

**No se necesita modificar esta sección**


**Random Forest sobre dataset con PCA**

In [ ]:
library(randomForest)
library(caret)

# Random Forest
RF_model <- randomForest(as.factor(trainingSet.OK) ~ ., data=trSet, method="class")
RF_predict <- predict(RF_model, teSet, type = "class")
confusionMatrix(RF_predict, as.factor(teSet$testingSet.OK), positive = '1')

